In [1]:
#@title Install dependencies
%%bash -s $use_amber $use_templates

set -e

USE_AMBER=$1
USE_TEMPLATES=$2

if [ ! -f COLABFOLD_READY ]; then
  # install dependencies
  # We have to use "--no-warn-conflicts" because colab already has a lot preinstalled with requirements different to ours
  pip install -q --no-warn-conflicts "colabfold[alphafold-minus-jax] @ git+https://github.com/sokrypton/ColabFold"
  # high risk high gain
  pip install -q "jax[cuda11_cudnn805]>=0.3.8,<0.4" -f https://storage.googleapis.com/jax-releases/jax_releases.html
  touch COLABFOLD_READY
fi

# setup conda
if [ ${USE_AMBER} == "True" ] || [ ${USE_TEMPLATES} == "True" ]; then
  if [ ! -f CONDA_READY ]; then
    wget -qnc https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local 2>&1 1>/dev/null
    rm Miniconda3-latest-Linux-x86_64.sh
    touch CONDA_READY
  fi
fi
# setup template search
if [ ${USE_TEMPLATES} == "True" ] && [ ! -f HH_READY ]; then
  conda install -y -q -c conda-forge -c bioconda kalign2=2.04 hhsuite=3.3.0 python=3.7 2>&1 1>/dev/null
  touch HH_READY
fi
# setup openmm for amber refinement
if [ ${USE_AMBER} == "True" ] && [ ! -f AMBER_READY ]; then
  conda install -y -q -c conda-forge openmm=7.5.1 python=3.7 pdbfixer 2>&1 1>/dev/null
  touch AMBER_READY
fi

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')

import shutil
if "alphafold_results" not in os.listdir("/content/drive/MyDrive"):
    os.mkdir("/content/drive/MyDrive/alphafold_results")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/alphafold_results/train.csv")
data["klen"] = data["protein_sequence"].apply(lambda x: len(x))
dataf = data[(data["klen"]>50) & (data["klen"]<200)].sort_values(by = "klen")
dataf

,seq_id,protein_sequence,pH,data_source,tm,klen
28315,28315,MVAYWRQAGLSYIRFSQICAKAVRDALKTEFKANAEKTSGSSIKIV...,7.0,doi.org/10.1038/s41592-020-0801-4,56.8,51
340,340,ALYVMRLALFNPDVSWDRKNNPEPWNKLGPNEQYKFYSVNVDYSKL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.9,52
19475,19475,MNQREQMEQQCQWRITVWEEENFQGKRCEFRLECPNILDRDFQKIR...,7.0,doi.org/10.1038/s41592-020-0801-4,52.0,52
1158,1158,ECSLVMDAKTGESKGFGFVVMPGPGNGKAAMKSLNATEIAGQIIRV...,7.0,doi.org/10.1038/s41592-020-0801-4,43.1,52
31251,31251,YCLFKKMHSSQDFRQTMSKKFPFILEVYYKSIEQSGMYGVREKDEE...,7.0,doi.org/10.1038/s41592-020-0801-4,52.4,52
...,...,...,...,...,...,...
9049,9049,MEHICGTSRIAGFRFSLYPMTDDFISVIKSALKKTDTSKVWTKTDH...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,199
9824,9824,MERTELLKPRTLADLIRILHELFAGDEVNVEEVQAVLEAYESNPAE...,7.0,doi.org/10.1038/s41592-020-0801-4,57.2,199
19175,19175,MNKPISEACLRNQGPIAAALKERLPNAARLLEIGSGTGQHAVFCAR...,7.0,doi.org/10.1038/s41592-020-0801-4,48.4,199
29660,29660,NLPTYLTIGAVAPTFAHLAAAKLVPIRGGPKEKEVVERNEQFTADS...,7.0,doi.org/10.1038/s41592-020-0801-4,35.7,199


In [2]:
list_sequences = list(data["protein_sequence"])[15:100]
# list_sequences

In [ ]:
#@title Input protein sequence(s), then hit `Runtime` -> `Run all`
from google.colab import files
from tqdm import tqdm
import os.path
import re
import hashlib
import random

def add_hash(x,y):
  return x+"_"+hashlib.sha1(y.encode()).hexdigest()[:5]


for i in tqdm(range(len(list_sequences))):

    
    print("########################################", i)
    # Online Python compiler (interpreter) to run Python online.
    query_sequence =  list_sequences[i]
    # remove whitespaces
    query_sequence = "".join(query_sequence.split())
    
    jobname = 'test' 
    # remove whitespaces
    basejobname = "".join(jobname.split())
    basejobname = re.sub(r'\W+', '', basejobname)
    jobname = add_hash(basejobname, query_sequence)
    while os.path.isfile(f"{jobname}.csv"):
      jobname = add_hash(basejobname, ''.join(random.sample(query_sequence,len(query_sequence))))
    
    with open(f"{jobname}.csv", "w") as text_file:
        text_file.write(f"id,sequence\n{jobname},{query_sequence}")
    
    queries_path=f"{jobname}.csv"
    
    # number of models to use
    use_amber = False
    template_mode = "none" 
    if template_mode == "pdb70":
      use_templates = True
      custom_template_path = None
    elif template_mode == "custom":
      custom_template_path = f"{jobname}_template"
      os.mkdir(custom_template_path)
      uploaded = files.upload()
      use_templates = True
      for fn in uploaded.keys():
        os.rename(fn, f"{jobname}_template/{fn}")
    else:
      custom_template_path = None
      use_templates = False
    
    
    
    #@title ### MSA options (custom MSA upload, single sequence, pairing mode)
    msa_mode = "MMseqs2 (UniRef+Environmental)" 
    pair_mode = "unpaired+paired" 
    
    # decide which a3m to use
    if msa_mode.startswith("MMseqs2"):
      a3m_file = f"{jobname}.a3m"
    elif msa_mode == "custom":
      a3m_file = f"{jobname}.custom.a3m"
      if not os.path.isfile(a3m_file):
        custom_msa_dict = files.upload()
        custom_msa = list(custom_msa_dict.keys())[0]
        header = 0
        import fileinput
        for line in fileinput.FileInput(custom_msa,inplace=1):
          if line.startswith(">"):
             header = header + 1
          if not line.rstrip():
            continue
          if line.startswith(">") == False and header == 1:
             query_sequence = line.rstrip()
          print(line, end='')
    
        os.rename(custom_msa, a3m_file)
        queries_path=a3m_file
        print(f"moving {custom_msa} to {a3m_file}")
    else:
      a3m_file = f"{jobname}.single_sequence.a3m"
      with open(a3m_file, "w") as text_file:
        text_file.write(">1\n%s" % query_sequence)
    
    
    
    
    
    #@title ### Advanced settings
    model_type = "auto" 
    num_recycles = 3 
    save_to_google_drive = False
    
    dpi = 200 
    
    if save_to_google_drive:
      from pydrive.drive import GoogleDrive
      from pydrive.auth import GoogleAuth
      from google.colab import auth
      from oauth2client.client import GoogleCredentials
      auth.authenticate_user()
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive = GoogleDrive(gauth)
      print("You are logged into Google Drive and are good to go!")
    
    #@title Run Prediction
    import sys
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    
    from colabfold.download import download_alphafold_params, default_data_dir
    from colabfold.utils import setup_logging
    from colabfold.batch import get_queries, run, set_model_type
    K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
    if "1" in K80_chk:
      print("WARNING: found GPU Tesla K80: limited to total length < 1000")
      if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
        del os.environ["TF_FORCE_UNIFIED_MEMORY"]
      if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
        del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]
    
    from colabfold.colabfold import plot_protein
    from pathlib import Path
    import matplotlib.pyplot as plt
    
    
    # For some reason we need that to get pdbfixer to import
    if use_amber and '/usr/local/lib/python3.7/site-packages/' not in sys.path:
        sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')
    
    # def prediction_callback(unrelaxed_protein, length, prediction_result, input_features, type):
    #   fig = plot_protein(unrelaxed_protein, Ls=length, dpi=150)
    #   plt.show()
    #   plt.close()
    
    result_dir="."
    if 'logging_setup' not in globals():
        setup_logging(Path(".").joinpath("log.txt"))
        logging_setup = True
    
    queries, is_complex = get_queries(queries_path)
    model_type = set_model_type(is_complex, model_type)
    download_alphafold_params(model_type, Path("."))
    run(
        queries=queries,
        result_dir=result_dir,
        use_templates=use_templates,
        custom_template_path=custom_template_path,
        use_amber=use_amber,
        msa_mode=msa_mode,    
        model_type=model_type,
        num_models=1,
        num_recycles=num_recycles,
        model_order=[1],
        is_complex=is_complex,
        data_dir=Path("."),
        keep_existing_results=False,
        recompile_padding=1.0,
        rank_by="auto",
        pair_mode=pair_mode,
        stop_at_score=float(100),
        # prediction_callback=prediction_callback,
        dpi=dpi
    )
    
    
    #@title Package and download results
    
    if msa_mode == "custom":
      print("Don't forget to cite your custom MSA generation method.")
    
    !zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png"
    # files.download(f"{jobname}.result.zip")
    shutil.copy(f"{jobname}.result.zip", "/content/drive/MyDrive/alphafold_results")
    
    
    # save_to_google_drive = False
    # if save_to_google_drive == True and drive:
    #   uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
    #   uploaded.SetContentFile(f"{jobname}.result.zip")
    #   uploaded.Upload()
    #   print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

  0%|          | 0/85 [00:00<?, ?it/s]

######################################## 0


  0%|          | 0/85 [00:00<?, ?it/s]

2022-11-10 16:06:35,862 Found 5 citations for tools or databases


  0%|          | 0/85 [00:05<?, ?it/s]

...

2022-11-10 19:19:28,571 Found 5 citations for tools or databases


 48%|████▊     | 41/85 [3:12:57<3:47:48, 310.64s/it]

2022-11-10 19:19:33,141 Query 1/1: test_1e281 (length 1417)



 48%|████▊     | 41/85 [3:13:05<3:47:48, 310.64s/it]

2022-11-10 19:19:41,180 Running model_1


In [ ]:
#@title ### MSA options (custom MSA upload, single sequence, pairing mode)
msa_mode = "MMseqs2 (UniRef+Environmental)" 
pair_mode = "unpaired+paired" 

# decide which a3m to use
if msa_mode.startswith("MMseqs2"):
  a3m_file = f"{jobname}.a3m"
elif msa_mode == "custom":
  a3m_file = f"{jobname}.custom.a3m"
  if not os.path.isfile(a3m_file):
    custom_msa_dict = files.upload()
    custom_msa = list(custom_msa_dict.keys())[0]
    header = 0
    import fileinput
    for line in fileinput.FileInput(custom_msa,inplace=1):
      if line.startswith(">"):
         header = header + 1
      if not line.rstrip():
        continue
      if line.startswith(">") == False and header == 1:
         query_sequence = line.rstrip()
      print(line, end='')

    os.rename(custom_msa, a3m_file)
    queries_path=a3m_file
    print(f"moving {custom_msa} to {a3m_file}")
else:
  a3m_file = f"{jobname}.single_sequence.a3m"
  with open(a3m_file, "w") as text_file:
    text_file.write(">1\n%s" % query_sequence)

In [ ]:
#@title ### Advanced settings
model_type = "auto" 
num_recycles = 3 
save_to_google_drive = False

dpi = 200 

if save_to_google_drive:
  from pydrive.drive import GoogleDrive
  from pydrive.auth import GoogleAuth
  from google.colab import auth
  from oauth2client.client import GoogleCredentials
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("You are logged into Google Drive and are good to go!")

In [ ]:
#@title Run Prediction
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from colabfold.download import download_alphafold_params, default_data_dir
from colabfold.utils import setup_logging
from colabfold.batch import get_queries, run, set_model_type
K80_chk = !nvidia-smi | grep "Tesla K80" | wc -l
if "1" in K80_chk:
  print("WARNING: found GPU Tesla K80: limited to total length < 1000")
  if "TF_FORCE_UNIFIED_MEMORY" in os.environ:
    del os.environ["TF_FORCE_UNIFIED_MEMORY"]
  if "XLA_PYTHON_CLIENT_MEM_FRACTION" in os.environ:
    del os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]

from colabfold.colabfold import plot_protein
from pathlib import Path
import matplotlib.pyplot as plt


# For some reason we need that to get pdbfixer to import
if use_amber and '/usr/local/lib/python3.7/site-packages/' not in sys.path:
    sys.path.insert(0, '/usr/local/lib/python3.7/site-packages/')

# def prediction_callback(unrelaxed_protein, length, prediction_result, input_features, type):
#   fig = plot_protein(unrelaxed_protein, Ls=length, dpi=150)
#   plt.show()
#   plt.close()

result_dir="."
if 'logging_setup' not in globals():
    setup_logging(Path(".").joinpath("log.txt"))
    logging_setup = True

queries, is_complex = get_queries(queries_path)
model_type = set_model_type(is_complex, model_type)
download_alphafold_params(model_type, Path("."))
run(
    queries=queries,
    result_dir=result_dir,
    use_templates=use_templates,
    custom_template_path=custom_template_path,
    use_amber=use_amber,
    msa_mode=msa_mode,    
    model_type=model_type,
    num_models=5,
    num_recycles=num_recycles,
    model_order=[1, 2, 3, 4, 5],
    is_complex=is_complex,
    data_dir=Path("."),
    keep_existing_results=False,
    recompile_padding=1.0,
    rank_by="auto",
    pair_mode=pair_mode,
    stop_at_score=float(100),
    # prediction_callback=prediction_callback,
    dpi=dpi
)

2022-11-01 05:43:54,619 Found 5 citations for tools or databases
2022-11-01 05:43:59,597 Query 1/1: test_29f79 (length 59)


SUBMIT:   0%|          | 0/150 [elapsed: 00:00 remaining: ?]


KeyboardInterrupt: ignored

In [ ]:
#@title Package and download results

if msa_mode == "custom":
  print("Don't forget to cite your custom MSA generation method.")

!zip -FSr $jobname".result.zip" config.json $jobname*".json" $jobname*".a3m" $jobname*"relaxed_rank_"*".pdb" "cite.bibtex" $jobname*".png"
# files.download(f"{jobname}.result.zip")
shutil.copy(f"{jobname}.result.zip", "/content/drive/MyDrive/alphafold_results")


# save_to_google_drive = False
# if save_to_google_drive == True and drive:
#   uploaded = drive.CreateFile({'title': f"{jobname}.result.zip"})
#   uploaded.SetContentFile(f"{jobname}.result.zip")
#   uploaded.Upload()
#   print(f"Uploaded {jobname}.result.zip to Google Drive with ID {uploaded.get('id')}")

Archive is current


'/content/drive/MyDrive/alphafold_results/test_2731e.result.zip'

In [ ]:
save_to_google_drive

False